# Example 30: PCA for dimensionality reduction

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Priiciapl Component Analysis](#ekf)
    * [Summary PCA](#sumekf)
    * [Test Case](#motion_model)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="overview"></a> Overview


In this notebook we will discuss <a href="https://en.wikipedia.org/wiki/Principal_component_analysis">Principal Component Analysis</a> for dimensionality reduction.

Dimensionality reduction refres to a number of techniques for reducing the dimensions associated with a data set. Consider, for example, a data set where each input point has five features. A dimensionality reduction technique can help us reduce the number of features to three or two. 

The primary reason behind why someone would like to reduce the number of dimensions of a data set is that a large number  of input features can cause poor performance of ML algorithms. This is frequently abbreviated as the curse of dimensionality. 

Furthermore, often fewer input dimensions translate to fewer model parameters or simpler model structure in general. A model with too many parameters is likely to overfit the training set and therefore it may not perform well on new data.

 Finally, dimensionality reduction may also be appropriate when the variables in a dataset are noisy.

### <a name="ekf"></a> Principal Component Analysis

PCA can be thought of as fitting a p-dimensional ellipsoid to the data. Each axis of the ellipsoid represents a principal component. If some axis of the ellipsoid is small, then the variance along that axis is also small, and by omitting that axis and its corresponding principal component from our representation of the dataset, we lose only an equally small amount of information.

So how can we find the axes of the ellipsoid? We first center the data around the origin by subtracting the mean of each variable from the dataset. Then, we compute the covariance matrix of the data and calculate the eigenvalues and corresponding eigenvectors of this covariance matrix. Then we must normalize each of the orthogonal eigenvectors to become unit vectors. Once this is done, each of the mutually orthogonal, unit eigenvectors can be interpreted as an axis of the ellipsoid fitted to the data. This choice of basis will transform our covariance matrix into a diagonalised form with the diagonal elements representing the variance of each axis. The proportion of the variance that each eigenvector represents can be calculated by dividing the eigenvalue corresponding to that eigenvector by the sum of all eigenvalues. 

PCA can be done either by performing a <a href="https://en.wikipedia.org/wiki/Singular_value_decomposition"> Singular Value Decomposition</a> or by doing the following two steps:

1. Calculate the data covariance (or correlation) matrix of the original data
2. Perform eigenvalue decomposition on the covariance matrix


These vectors  should be eignevectors of the matrix $\mathbf{X}^T\mathbf{X}$. Thus they should satisfy the following equation:

$$\mathbf{X}^T  \mathbf{X}\mathbf{w}_j = \lambda_j\mathbf{w}_{j}$$

The matrix $\mathbf{X}^T  \mathbf{X}$ is symmetric and therefore the eigenvectors are orthogonal.

### Singular Value Decomposition

The SVD of a matrix $\mathbf{P}$ has the following form

$$\mathbf{P} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^T$$

where $\mathbf{\Sigma}$ is  a diagonal-like matrix containing the singular values $\sigma_i$ and $\mathbf{V}$ is an orthogonal matrix. The eigenvalues $\lambda_i$ of $\mathbf{P}^T\mathbf{P}$ and the singular values $\sigma_i$ of $\mathbf{P}$ are connected via

$$\lambda_i = \sigma_{i}^2$$

Furthermore, the columns of $\mathbf{V}$ are the eigenvectors for $\mathbf{P}^T\mathbf{P}$. Moreover, the SVD process orders the singular values according to their size: $\sigma_1 \ge \sigma_2 \ge \dots \ge 0$

Hence, we can use as $\mathbf{w}_1$ the first column of $\mathbf{V}$ as this will be an eigenvector for the largest singular value $\sigma_1 = \sqrt{\lambda_1}$. Similarly for the rest $\mathbf{w}_i$. The columns of $\mathbf{V}$ are called the principal components.

The total variance is the sum of variances of all individual principal components. The fraction of variance explained by a principal component is the ratio between the variance of that principal component and the total variance.

### <a name="sumekf"></a> Summary of PCA

In summary, the first $k$ principal components (where can be 1, 2, 3 etc.) explain the most variance any $k$ variables can explain, and the last $k$ variables explain the least variance any variables can explain, under some general restrictions. (The restrictions ensure, for example, that we cannot adjust a variable’s explained variance simply by scaling it.)

### <a name="motion_model"></a> Test Case

Let us first consider the following toy example. The data set $\mathbf{X}$ is:

```
DynMat<real_t> X(6, 2);
X(0,0) = -1.;
X(0,1) = -1.;
    
X(1,0) = -2.;
X(1,1) = -1.;
    
X(2,0) = -3.;
X(2,1) = -2.;
    
X(3,0) = 1.;
X(3,1) = 1.;
    
X(4,0) = 2.;
X(4,1) = 1.;
    
X(5,0) = 3.;
X(5,1) = 2.;
```

Observe that the empirical mean for each of the two columns is zero. The <a href="https://bitbucket.org/blaze-lib/blaze/src/master/">Blaze</a> library that we use to represent matrices and vectors has support for SVD. We will use the following function in the code below


```
template< typename MT1, bool SO, typename VT, bool TF, typename MT2, typename MT3 >
void svd( const DenseMatrix<MT1,SO>& A, DenseMatrix<MT2,SO>& U,
          DenseVector<VT,TF>& s, DenseMatrix<MT3,SO>& V );
```

## <a name="include_files"></a> Include files

```
#include "cubic_engine/base/cubic_engine_types.h"
#include "kernel/maths/matrix_utilities.h"
#include <cmath>
#include <iostream>
#include <tuple>
```

## <a name="m_func"></a> The main function

```
namespace example
{

using cengine::uint_t;
using cengine::real_t;
using cengine::DynMat;
using cengine::DynVec;

}

int main() {
   
    using namespace example;
    
    DynMat<real_t> X(6, 2);
    X(0,0) = -1.;
    X(0,1) = -1.;
    
    X(1,0) = -2.;
    X(1,1) = -1.;
    
    X(2,0) = -3.;
    X(2,1) = -2.;
    
    X(3,0) = 1.;
    X(3,1) = 1.;
    
    X(4,0) = 2.;
    X(4,1) = 1.;
    
    X(5,0) = 3.;
    X(5,1) = 2.;

    // caluclate the sample variance
    // of each of the 3 variables (columns)
    auto col1 = kernel::get_column(X, 0);
    auto col2 = kernel::get_column(X, 1);

    auto col1_var = var(col1);
    auto col2_var = var(col2);

    std::cout<<"Variable 1 variance: "<<col1_var<<std::endl;
    std::cout<<"Variable 2 variance: "<<col2_var<<std::endl;

    // compute the total variance
    auto total_var = col1_var + col2_var;

    std::cout<<"Total variance: "<<total_var<<std::endl;

    DynMat<real_t> U;
    DynVec<real_t> s;
    DynMat<real_t> V;
    try{

        std::cout<<"Variable 1 explains: "<<col1_var/total_var<<std::endl;
        std::cout<<"Variable 2 explains: "<<col2_var/total_var<<std::endl;

        svd(X, U, s, V );

        std::cout<<"Singular values: "<<s<<std::endl;

        // Principal axes in feature space,
        // representing the directions of maximum variance in the data.
        // these are the columns of the V matrix
        std::cout<<"V matrix: "<<V<<std::endl;

        // reconstruct the data set with PCA
        // The full principal components decomposition of
        // X can be given as T= XW
        DynMat<real_t> T = X*V;

        // caluclate the sample variance
        // of each of the 3 variables (columns)
        auto pca_col1 = kernel::get_column(T, 0);
        auto pca_col2 = kernel::get_column(T, 1);

        auto pca_col1_var = var(pca_col1);
        auto pca_col2_var = var(pca_col2);

        std::cout<<"PCA variable 1 variance: "<<pca_col1_var<<std::endl;
        std::cout<<"PCA variable 2 variance: "<<pca_col2_var<<std::endl;

        // this should be the same at the total variance
        // compute the total variance
        auto pca_total_var = pca_col1_var + pca_col2_var;

        std::cout<<"PCA Total variance: "<<pca_total_var<<std::endl;

        std::cout<<"PCA Variable 1 explains: "<<pca_col1_var/pca_total_var<<std::endl;
        std::cout<<"PCA Variable 2 explains: "<<pca_col2_var/pca_total_var<<std::endl;


    }
    catch(std::runtime_error& e){
        std::cerr<<"Runtime error: "
                 <<e.what()<<std::endl;
    }
    catch(std::logic_error& e){
        std::cerr<<"Logic error: "
                 <<e.what()<<std::endl;
    }
    catch(...){
        std::cerr<<"Unknown exception was raised whilst running simulation."<<std::endl;
    }
   
    return 0;
}

```

## <a name="results"></a> Results


Upon running the driver code above we get:

```
Variable 1 variance: 5.6
Variable 2 variance: 2.4
Total variance: 8
Variable 1 explains: 0.7
Variable 2 explains: 0.3
Singular values: (     6.30061 )
(    0.549804 )

V matrix: (    -0.838492    -0.544914 )
(    -0.544914     0.838492 )

PCA variable 1 variance: 7.93954
PCA variable 2 variance: 0.0604569
PCA Total variance: 8
PCA Variable 1 explains: 0.992443
PCA Variable 2 explains: 0.00755711

```

## <a name="source_code"></a> Source Code


<a href="../exe.cpp">exe.cpp</a>